# Torch.Autograd를 사용한 자동미분

역전파 알고리즘, 매개변수(모델 가중치)는 주어진 매개변수에 대한 손실함수의 gradient에 따라 조정됨.

torch.autograd 를 통해 모든 계산 그래프에 대한 gradient의 자동 계산을 지원

* x : input
* w, b : parameter
단일 계층 신경망 가정


In [1]:
import torch


x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## Tensor, Function과 연산 그래프

In [2]:
print(f'Gradient function for z={z.grad_fn}')
print(f'Gradient function for loss = {loss.grad_fn}')

Gradient function for z=<AddBackward0 object at 0x000001B5139F5B70>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x000001B5137DEFB0>


## Gradient 계산하기

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1168, 0.0445, 0.2100],
        [0.1168, 0.0445, 0.2100],
        [0.1168, 0.0445, 0.2100],
        [0.1168, 0.0445, 0.2100],
        [0.1168, 0.0445, 0.2100]])
tensor([0.1168, 0.0445, 0.2100])


### Gradient 추적 멈추기

모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 순전파 연산만 필요한 경우에는 torch.no_grad() 블록으로 둘러싸서 연산 추적을 멈출 수 있다.

In [4]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

True


In [5]:
with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

False


In [6]:
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


gradient 추적을 멈춰야 하는 경우

* 신경망의 일부 매개변수를 frozen parameter 로 표시
* 변화도를 추적하지 않는 텐서의 연산이 더 효율적, 순전파 단계만 수행할 때 연산속도가 향상

Loss function을 가지고 gradient를 계산하지만 출력함수가 임의의 텐서인 경우 실제 변화도가 아닌 Jacobian product를 계산한다.

In [7]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f'First call \n {inp.grad}')

out.backward(torch.ones_like(out), retain_graph=True)
print(f'\n Second call \n{inp.grad}')

inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f'\ncall after zeroing gradients \n{inp.grad}')

First call 
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

 Second call 
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

call after zeroing gradients 
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


동일한 인자로 backward를 두차례 호출하면 변화도 값이 달라짐.
역방향 전파를 수행할 때 변화도를 accumulate 해주기 때문 